In [1]:
import akshare as ak
from typing import Optional


In [16]:
from pydantic import BaseModel

# Pydantic models for request and response validation
class FinancialAbstract(BaseModel):
    reporting_period: object
    net_profit: Optional[object] = None
    net_profit_growth_rate: Optional[object] = None
    non_recurring_net_profit: Optional[object] = None
    non_recurring_net_profit_growth_rate: Optional[object] = None
    total_operating_revenue: Optional[object] = None
    total_operating_revenue_growth_rate: Optional[object] = None
    basic_earnings_per_share: Optional[object] = None
    net_asset_per_share: Optional[object] = None
    capital_reserve_fund_per_share: Optional[object] = None
    undistributed_profit_per_share: Optional[object] = None
    operating_cash_flow_per_share: Optional[object] = None
    net_profit_margin: Optional[object] = None
    gross_profit_margin: Optional[object] = None
    return_on_equity_of_roe: Optional[object] = None
    diluted_return_on_equity_of_roe: Optional[object] = None
    operating_cycle: Optional[object] = None
    inventory_turnover_ratio: Optional[object] = None
    days_inventory_outstanding: Optional[object] = None
    days_sales_outstanding: Optional[object] = None
    current_ratio: Optional[object] = None
    quick_ratio: Optional[object] = None
    conservative_quick_ratio : Optional[object] = None
    debt_to_equity_ratio: Optional[object] = None
    asset_to_liability_ratio: Optional[object] = None

def get_stock_financial_abstract(stock_code: str ,
                                 indicator: Optional[str] = '按报告期') -> list:
    
        
        result = ak.stock_financial_abstract_ths(symbol=stock_code, indicator=indicator)
        column_mapping = {
            "报告期" : "reporting_period",
            "净利润" : "net_profit",
            "净利润同比增长率" : "net_profit_growth_rate",
            "扣非净利润" : "non_recurring_net_profit",
            "扣非净利润同比增长率" : "non_recurring_net_profit_growth_rate",
            "营业总收入" : "total_operating_revenue",
            "营业总收入同比增长率" : "total_operating_revenue_growth_rate",
            "基本每股收益" : "basic_earnings_per_share",
            "每股净资产" : "net_asset_per_share",
            "每股资本公积金" : "capital_reserve_fund_per_share" ,
            "每股未分配利润" : "undistributed_profit_per_share" ,
            "每股经营现金流" : "operating_cash_flow_per_share" ,
            "销售净利率" : "net_profit_margin" ,
            "销售毛利率" : "gross_profit_margin",
            "净资产收益率" : "return_on_equity_of_roe" ,
            "净资产收益率-摊薄" : "diluted_return_on_equity_of_roe" ,
            "营业周期" : "operating_cycle" ,
            "存货周转率" : "inventory_turnover_ratio" ,
            "存货周转天数" : "days_inventory_outstanding" ,
            "应收账款周转天数" : "days_sales_outstanding" ,
            "流动比率" : "current_ratio" ,
            "速动比率" : "quick_ratio" ,
            "保守速动比率" : "conservative_quick_ratio" ,
            "产权比率" : "debt_to_equity_ratio" ,
            "资产负债率" : "asset_to_liability_ratio" ,
        }
        result.rename(columns=column_mapping, inplace=True)
        financial_abstracts = result.apply(lambda row: FinancialAbstract(**row).model_dump(), axis=1).tolist()
        print(financial_abstracts)


In [17]:
get_stock_financial_abstract(stock_code='000001')



[{'reporting_period': '1989-12-31', 'net_profit': '4302.00万', 'net_profit_growth_rate': False, 'non_recurring_net_profit': False, 'non_recurring_net_profit_growth_rate': False, 'total_operating_revenue': False, 'total_operating_revenue_growth_rate': False, 'basic_earnings_per_share': False, 'net_asset_per_share': '2.97', 'capital_reserve_fund_per_share': False, 'undistributed_profit_per_share': '1.46', 'operating_cash_flow_per_share': False, 'net_profit_margin': False, 'gross_profit_margin': None, 'return_on_equity_of_roe': False, 'diluted_return_on_equity_of_roe': '29.86%', 'operating_cycle': False, 'inventory_turnover_ratio': None, 'days_inventory_outstanding': None, 'days_sales_outstanding': False, 'current_ratio': '9.61', 'quick_ratio': '9.61', 'conservative_quick_ratio': '9.61', 'debt_to_equity_ratio': '0.28', 'asset_to_liability_ratio': '2.37%'}, {'reporting_period': '1990-12-31', 'net_profit': '7087.50万', 'net_profit_growth_rate': '64.75%', 'non_recurring_net_profit': False, 'no

In [ ]:
from typing import Any, Dict, List, Optional, Union
import pandas as pd
from typing import List, Dict, Any
import pandas as pd
import akshare as ak

def get_stock_margin_detail(stock_code: str, start_date: str, end_date: str, freq: str = "D") -> List[Dict[str, Any]]:
    """
    获取中国 A 股上市公司的融资融券明细数据。

    Args:
        stock_code: 股票代码，例如 "600000"。
        start_date: 开始日期，格式为 YYYYMMDD。
        end_date: 结束日期，格式为 YYYYMMDD。
        freq: 日期间隔类型，默认为 "D"（每日）。可选值包括：
                    - "D": 每日
                    - "W": 每周
                    - "MS": 每月的第一天
                    - "ME": 每月的最后一天
                    - "Q": 每季度
                    - "Y": 每年

    Returns:
        List[Dict[str, Any]]: 返回字典的列表，每个字典表示一个交易日的融资融券概要。
        每条记录包含以下字段：
        | 字段名                 | 数据类型   | 描述                           |
        |------------------------|------------|--------------------------------|
        | trading_date           | str        | 交易日期                       |
        | target_security_code   | str        | 标的证券代码                   |
        | target_security_name   | str        | 标的证券简称                   |
        | margin_balance         | int        | 融资余额 (单位: 元)             |
        | margin_buy_amount      | int        | 融资买入额 (单位: 元)           |
        | margin_repayment       | int        | 融资偿还额 (单位: 元)           |
        | short_selling_balance  | int        | 融券余量                       |
        | short_selling_volume   | int        | 融券卖出量                     |
        | short_selling_repayment| int        | 融券偿还量                     |

    Notes:
        - 如果某一天没有数据或发生异常，会跳过该日期并继续处理后续日期。
        - 返回的数据已将中文字段名翻译为英文，便于与国际化系统集成。
    """
    # 初始化空的 DataFrame 用于存储所有数据
    all_filtered_df = pd.DataFrame()

    # 定义字段映射（中文字段名 -> 英文字段名）
    column_mapping = {
        "信用交易日期": "trading_date",
        "标的证券代码": "target_security_code",
        "标的证券简称": "target_security_name",
        "融资余额": "margin_balance",
        "融资买入额": "margin_buy_amount",
        "融资偿还额": "margin_repayment",
        "融券余量": "short_selling_balance",
        "融券卖出量": "short_selling_volume",
        "融券偿还量": "short_selling_repayment",
    }

    # 将日期范围转换为日期列表
    for date in pd.date_range(start=start_date, end=end_date, freq=freq):
        # 格式化日期为字符串
        data_date = date.strftime('%Y%m%d')

        try:
            # 获取当天的融资融券数据
            stock_margin_detail_sse_df = ak.stock_margin_detail_sse(date=data_date)

            # 检查返回的数据是否为空
            if stock_margin_detail_sse_df is None or stock_margin_detail_sse_df.empty:
                print(f"No data for date: {data_date}")
                continue  # 跳过没有数据的日期

            # 过滤证券代码为指定的 stock_code
            filtered_df = stock_margin_detail_sse_df[
                stock_margin_detail_sse_df['标的证券代码'] == stock_code
            ].copy()

            # 再次检查过滤后的数据是否为空
            if filtered_df is None or filtered_df.empty:
                print(f"No data for stock code {stock_code} on date: {data_date}")
                continue  # 跳过没有匹配数据的日期

            # 翻译字段名为英文
            filtered_df.rename(columns=column_mapping, inplace=True)

            # 将当天数据添加到总数据中
            all_filtered_df = pd.concat([all_filtered_df, filtered_df], ignore_index=True)

        except Exception as e:
            print(f"Error fetching data for date {data_date}: {e}")
            continue  # 跳过发生错误的日期

    # 将 DataFrame 转换为字典列表
    result = all_filtered_df.to_dict(orient="records")

    return result
get_stock_margin_detail(stock_code="688799", start_date='20240101', end_date='20241231',freq='MS')  

C:\Users\bigdata\AppData\Local\Temp\ipykernel_24632\1502098118.py:58: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  for date in pd.date_range(start=start_date, end=end_date, freq=freq):


Error fetching data for date 20240331: Length mismatch: Expected axis has 0 elements, new values have 13 elements
Error fetching data for date 20240630: Length mismatch: Expected axis has 0 elements, new values have 13 elements
Error fetching data for date 20240831: Length mismatch: Expected axis has 0 elements, new values have 13 elements
Error fetching data for date 20241130: Length mismatch: Expected axis has 0 elements, new values have 13 elements


[{'trading_date': '20240131',
  'target_security_code': '688799',
  'target_security_name': '华纳药厂',
  'margin_balance': 172179693,
  'margin_buy_amount': 3752975,
  'margin_repayment': 2175779,
  'short_selling_balance': 7275,
  'short_selling_volume': 1200,
  'short_selling_repayment': 1500},
 {'trading_date': '20240229',
  'target_security_code': '688799',
  'target_security_name': '华纳药厂',
  'margin_balance': 175176701,
  'margin_buy_amount': 8356871,
  'margin_repayment': 2264409,
  'short_selling_balance': 8934,
  'short_selling_volume': 700,
  'short_selling_repayment': 0},
 {'trading_date': '20240430',
  'target_security_code': '688799',
  'target_security_name': '华纳药厂',
  'margin_balance': 193543348,
  'margin_buy_amount': 11416492,
  'margin_repayment': 26807362,
  'short_selling_balance': 5918,
  'short_selling_volume': 1900,
  'short_selling_repayment': 200},
 {'trading_date': '20240531',
  'target_security_code': '688799',
  'target_security_name': '华纳药厂',
  'margin_balance'